In [7]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.utils.multiclass import type_of_target
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from bayes_opt import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
import warnings

# Correct paths
weather_data_path = '/Users/andrewfearney27/Downloads/df_cleaned.csv'
pleasant_weather_path = '/Users/andrewfearney27/Downloads/Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'

# Importing data
unscaled = pd.read_csv(weather_data_path, index_col=False)
pleasant = pd.read_csv(pleasant_weather_path, index_col=False)

In [15]:
unscaled.shape

(22950, 136)

In [16]:
pleasant.shape


(22950, 16)

In [19]:
# Data Wrangling
# Checking initial shape and columns of the datasets
print(f"Initial unscaled shape: {unscaled.shape}")
print(f"Initial pleasant shape: {pleasant.shape}")

# Columns to drop due to irrelevance or insufficient data
columns_to_drop = [
    'GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation',
    'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
    'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure',
    'ROMA_sunshine', 'ROMA_temp_mean', 'ROMA_temp_min', 'ROMA_temp_max',
    'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_global_radiation',
    'TOURS_precipitation', 'TOURS_pressure', 'TOURS_temp_min', 'TOURS_temp_max'
]

# Dropping the unnecessary columns from the unscaled data
unscaled.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Further columns to drop related to wind speed and snow depth, as they lack sufficient data
further_drops = [
    'BASEL_snow_depth', 'DUSSELDORF_snow_depth', 'HEATHROW_snow_depth',
    'MUNCHENB_snow_depth', 'OSLO_snow_depth', 'VALENTIA_snow_depth',
    'BASEL_wind_speed', 'DEBILT_wind_speed', 'DUSSELDORF_wind_speed',
    'KASSEL_wind_speed', 'LJUBLJANA_wind_speed', 'MAASTRICHT_wind_speed',
    'MADRID_wind_speed', 'OSLO_wind_speed', 'SONNBLICK_wind_speed'
]

# Dropping further columns with limited data
unscaled.drop(columns=further_drops, inplace=True, errors='ignore')

# Inserting dummy values for columns that are missing data
# These columns will only be added if they are not already present
if 'KASSEL_cloud_cover' not in unscaled.columns:
    unscaled.insert(54, 'KASSEL_cloud_cover', unscaled['LJUBLJANA_cloud_cover'])

if 'MUNCHENB_pressure' not in unscaled.columns:
    unscaled.insert(92, 'MUNCHENB_pressure', unscaled['SONNBLICK_pressure'])

if 'STOCKHOLM_humidity' not in unscaled.columns:
    unscaled.insert(118, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])

# Final check on columns and shape
print(f"Final columns: {unscaled.columns.tolist()}")
print(f"Final unscaled shape: {unscaled.shape}")
print(f"Final pleasant shape: {pleasant.shape}")


Initial unscaled shape: (22950, 136)
Initial pleasant shape: (22950, 15)
Final columns: ['Unnamed: 0', 'BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max', 'BELGRADE_cloud_cover', 'BELGRADE_humidity', 'BELGRADE_pressure', 'BELGRADE_global_radiation', 'BELGRADE_precipitation', 'BELGRADE_sunshine', 'BELGRADE_temp_mean', 'BELGRADE_temp_min', 'BELGRADE_temp_max', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min', 'BUDAPEST_temp_max', 'DEBILT_cloud_cover', 'DEBILT_humidity', 'DEBILT_pressure', 'DEBILT_global_radiation', 'DEBILT_precipitation', 'DEBILT_sunshine', 'DEBILT_temp_mean', 'DEBILT_temp_min', 'DEBILT_temp_max', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_radiation', 'DUSSELDORF_pre

In [26]:
# Step 1: Check the initial shapes
print(f"Initial shape of X: {X.shape}")
print(f"Initial shape of y: {y.shape}")

# Step 2: Ensure X and y have the correct dimensions
# Since X.shape[0] should match y.shape[0], we'll truncate X if necessary
if X.shape[0] != y.shape[0]:
    X = X[:y.shape[0]]
    print(f"Truncated X to match y: {X.shape}")


Initial shape of X: (22950, 15, 9)
Initial shape of y: (22950,)


In [29]:
#Step 3: Reshape X into the required form
X = X.reshape(y.shape[0], 15, 9)
print(f"Reshaped X: {X.shape}")

# Step 4: Verify the shapes of X and y
print(f"Final shape of X: {X.shape}")
print(f"Final shape of y: {y.shape}")

# Step 5: Convert X and y from a DataFrame to NumPy arrays (if not already)
X = np.array(X)
y = np.array(y)

# Step 6: Verify the types and shapes
print(f"Type of X: {type(X)} with shape {X.shape}")
print(f"Type of y: {type(y)} with shape {y.shape}")

# Proceed without the argmax step



Reshaped X: (22950, 15, 9)
Final shape of X: (22950, 15, 9)
Final shape of y: (22950,)
Type of X: <class 'numpy.ndarray'> with shape (22950, 15, 9)
Type of y: <class 'numpy.ndarray'> with shape (22950,)


In [30]:
# Ensure that X has the same number of samples as y before reshaping
X = X[:y.shape[0]]  # Truncate X to match the length of y

# Reshape X to the correct shape
X = X.reshape(y.shape[0], 15, 9)

# Now split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)

# Print the shapes to verify everything is correct
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (18360, 15, 9)
Shape of X_test: (4590, 15, 9)
Shape of y_train: (18360,)
Shape of y_test: (4590,)


In [35]:
from sklearn.utils.multiclass import type_of_target
print(type_of_target(y))


binary


In [37]:
# Define scorer
score_acc = make_scorer(accuracy_score)

# Create the function for the Bayesian search
def bayesian_search(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs, layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential']

    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(X_train.shape[1], X_train.shape[2])))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for _ in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate))
        for _ in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(15, activation='softmax'))  # Assuming 15 classes
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

    model = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=39)
    score = cross_val_score(model, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)]}).mean()
    return score

# Define the parameter space
params = {
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation': (0, 7),  # Indices for activation functions
    'optimizer': (0, 7),  # Indices for optimizers
    'learning_rate': (0.001, 0.1),
    'batch_size': (100, 500),
    'epochs': (10, 100),
    'layers1': (1, 3),
    'layers2': (1, 3),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0.1, 0.5)
}

# Step 4: Run Bayesian Optimization with proper verbosity and iterations
start = time.time()

nn_opt = BayesianOptimization(f=bayesian_search, pbounds=params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=10)

print('Search took %s minutes' % ((time.time() - start)/60))

# Extract best parameters and score
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/24


/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 1s - 39ms/step - accuracy: 0.9700 - loss: 2.6993
Epoch 2/24
31/31 - 1s - 22ms/step - accuracy: 1.0000 - loss: 2.6899
Epoch 3/24
31/31 - 1s - 22ms/step - accuracy: 1.0000 - loss: 2.6827
Epoch 4/24
31/31 - 1s - 22ms/step - accuracy: 1.0000 - loss: 2.6765
Epoch 5/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6708
Epoch 6/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6652
Epoch 7/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6595
Epoch 8/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6533
Epoch 9/24
31/31 - 1s - 24ms/step - accuracy: 1.0000 - loss: 2.6465
Epoch 10/24
31/31 - 1s - 24ms/step - accuracy: 1.0000 - loss: 2.6386
Epoch 11/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6292
Epoch 12/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6179
Epoch 13/24
31/31 - 1s - 24ms/step - accuracy: 1.0000 - loss: 2.6043
Epoch 14/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.5879
Epoch 15/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 

/opt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 1s - 40ms/step - accuracy: 0.9677 - loss: 2.7068
Epoch 2/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6901
Epoch 3/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6829
Epoch 4/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6769
Epoch 5/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6713
Epoch 6/24
31/31 - 1s - 24ms/step - accuracy: 1.0000 - loss: 2.6660
Epoch 7/24
31/31 - 1s - 24ms/step - accuracy: 1.0000 - loss: 2.6607
Epoch 8/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6550
Epoch 9/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6488
Epoch 10/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6417
Epoch 11/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6332
Epoch 12/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6231
Epoch 13/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.6108
Epoch 14/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 - loss: 2.5960
Epoch 15/24
31/31 - 1s - 23ms/step - accuracy: 1.0000 

...

/opt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44/44 - 1s - 18ms/step - accuracy: 0.8760 - loss: 0.7648
Epoch 2/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 2.4641e-04
Epoch 3/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.4728e-04
Epoch 4/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.2874e-04
Epoch 5/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.1284e-04
Epoch 6/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 9.9387e-05
Epoch 7/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 8.6885e-05
Epoch 8/56
44/44 - 0s - 8ms/step - accuracy: 1.0000 - loss: 7.4063e-05
Epoch 9/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 6.5323e-05
Epoch 10/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 5.7917e-05
Epoch 11/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 5.1679e-05
Epoch 12/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 4.5958e-05
Epoch 13/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 4.1401e-05
Epoch 14/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 3.7486e-05
Epoch 15/56
44/

/opt/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44/44 - 1s - 18ms/step - accuracy: 0.8282 - loss: 0.7761
Epoch 2/56
44/44 - 0s - 6ms/step - accuracy: 1.0000 - loss: 4.6148e-04
Epoch 3/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 2.6492e-04
Epoch 4/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.9718e-04
Epoch 5/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.4402e-04
Epoch 6/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 1.1144e-04
Epoch 7/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 8.7857e-05
Epoch 8/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 7.0809e-05
Epoch 9/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 5.7185e-05
Epoch 10/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 4.6501e-05
Epoch 11/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 3.8501e-05
Epoch 12/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 3.1825e-05
Epoch 13/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 2.6299e-05
Epoch 14/56
44/44 - 0s - 7ms/step - accuracy: 1.0000 - loss: 2.1959e-05
Epoch 15/56
44/

ValueError: Input y contains NaN.

In [41]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']
print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")


Best Parameters: {'activation': 5.827098485602952, 'batch_size': 184.93564427131045, 'dropout': 0.18182496720710062, 'dropout_rate': 0.17336180394137354, 'epochs': 37.381801866358394, 'kernel': 2.049512863264476, 'layers1': 1.8638900372842315, 'layers2': 1.5824582803960838, 'learning_rate': 0.06157343657751557, 'neurons': 22.554447458683764, 'normalization': 0.29214464853521815, 'optimizer': 2.564532903055842}
Highest Accuracy: nan


In [43]:
# Interpret the best parameters and convert indices to actual names where needed
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU']
optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

best_params['activation'] = activationL[int(round(best_params['activation']))]
best_params['optimizer'] = optimizerL[int(round(best_params['optimizer']))]
best_params['neurons'] = int(round(best_params['neurons']))
best_params['kernel'] = int(round(best_params['kernel']))
best_params['batch_size'] = int(round(best_params['batch_size']))
best_params['epochs'] = int(round(best_params['epochs']))
best_params['layers1'] = int(round(best_params['layers1']))
best_params['layers2'] = int(round(best_params['layers2']))

print(f"Best Parameters (interpreted): {best_params}")

# Update the CNN model with the best parameters from Bayesian optimization
def optimized_cnn_model():
    model = Sequential()
    model.add(Conv1D(
        filters=best_params['neurons'],
        kernel_size=best_params['kernel'],
        activation=best_params['activation'],
        input_shape=(X_train.shape[1], X_train.shape[2])
    ))
    
    if best_params['normalization'] > 0.5:
        model.add(BatchNormalization())
    
    for _ in range(best_params['layers1']):
        model.add(Dense(best_params['neurons'], activation=best_params['activation']))
    
    if best_params['dropout'] > 0.5:
        model.add(Dropout(best_params['dropout_rate']))
    
    for _ in range(best_params['layers2']):
        model.add(Dense(best_params['neurons'], activation=best_params['activation']))
    
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(15, activation='softmax'))  # Assuming 15 classes
    
    optimizer = best_params['optimizer']
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

Best Parameters (interpreted): {'activation': 'elu', 'batch_size': 185, 'dropout': 0.18182496720710062, 'dropout_rate': 0.17336180394137354, 'epochs': 37, 'kernel': 2, 'layers1': 2, 'layers2': 2, 'learning_rate': 0.06157343657751557, 'neurons': 23, 'normalization': 0.29214464853521815, 'optimizer': 'Adadelta'}


In [46]:
# Extract the best parameters from the optimization
best_params = {
    'neurons': 61,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adadelta',
    'learning_rate': 0.76317719813,
    'batch_size': 460,
    'epochs': 91,
    'layers1': 1,
    'layers2': 2,
    'normalization': 0.77096717995,
    'dropout': 0.72960617833,
    'dropout_rate': 0.19126724141
}

# Initialize optimizer with learning rate
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}
optimizer = optimizers[best_params['optimizer']]


In [48]:
#Train the CNN model with the optimized parameters
model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(X_train.shape[1], X_train.shape[2])))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate']))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(15, activation='softmax'))  # Assuming 15 classes

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with the optimised parameters
history = model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)


Epoch 1/91
40/40 - 1s - 25ms/step - accuracy: 0.8706 - loss: 0.7981
Epoch 2/91
40/40 - 1s - 13ms/step - accuracy: 1.0000 - loss: 0.0120
Epoch 3/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 0.0041
Epoch 4/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 0.0019
Epoch 5/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 8.9293e-04
Epoch 6/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 4.5774e-04
Epoch 7/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 2.5125e-04
Epoch 8/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 1.5824e-04
Epoch 9/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 1.1269e-04
Epoch 10/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 8.7050e-05
Epoch 11/91
40/40 - 1s - 13ms/step - accuracy: 1.0000 - loss: 7.0719e-05
Epoch 12/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 5.9893e-05
Epoch 13/91
40/40 - 1s - 15ms/step - accuracy: 1.0000 - loss: 5.0395e-05
Epoch 14/91
40/40 - 1s - 14ms/step - accuracy: 1.0000 - loss: 4.4249e-05
Epo

In [49]:
# Display model summary
model.summary()

Model: "sequential_176"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_176 (Conv1D)             │ (None, 14, 61)         │         1,159 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_66          │ (None, 14, 61)         │           244 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_889 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_106 (Dropout)           │ (None, 14, 61)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_890 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_891 (Dense)               │ (None, 14, 61)         │         3,782 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_176               │ (None, 7, 61)          │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_176 (Flatten)           │ (None, 427)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_892 (Dense)               │ (None, 15)             │         6,420 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,265 (223.70 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

 Optimizer params: 38,096 (148.82 KB)

In [75]:
best_params = {
    'neurons': 61,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adadelta',
    'learning_rate': 0.76317719813,
    'batch_size': 460,
    'epochs': 91,
    'layers1': 1,
    'layers2': 1,
    'normalization': 0.77096717995,
    'dropout': 0.72966017833,
    'dropout_rate': 0.19126724140656393
}


In [76]:
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]


In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout, BatchNormalization

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], 
                 activation=best_params['activation'], input_shape=(15, 9)))

# Add normalization if required
if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

# Add dense layers as per layers1
for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

# Add dropout if required
if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate']))

# Add more dense layers as per layers2
for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(15, activation='softmax'))  # Assuming 15 classes

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [78]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=15)  # Adjust class count if necessary
y_test_one_hot = to_categorical(y_test, num_classes=15)

# Check shapes to ensure everything is correct
print(f"X_train shape: {X_train.shape}")
print(f"y_train_one_hot shape: {y_train_one_hot.shape}")


X_train shape: (18360, 15, 9)
y_train_one_hot shape: (18360, 15)


In [79]:
history = model.fit(X_train, y_train_one_hot, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=2)


Epoch 1/91
40/40 - 1s - 21ms/step - accuracy: 0.8901 - loss: 0.7824
Epoch 2/91
40/40 - 0s - 10ms/step - accuracy: 1.0000 - loss: 0.0248
Epoch 3/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.0075
Epoch 4/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.0031
Epoch 5/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 0.0014
Epoch 6/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 6.8278e-04
Epoch 7/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 3.5824e-04
Epoch 8/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 2.1423e-04
Epoch 9/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 1.4642e-04
Epoch 10/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 1.1096e-04
Epoch 11/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 8.8794e-05
Epoch 12/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 7.3262e-05
Epoch 13/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 6.3311e-05
Epoch 14/91
40/40 - 0s - 11ms/step - accuracy: 1.0000 - loss: 5.4902e-05
Epoch 1

In [81]:
y_pred = model.predict(X_test)


144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step


In [88]:
stations = {
    0: 'BASEL',
    1: 'BELGRADE',
    2: 'BUDAPEST',
    3: 'DEBILT',
    4: 'DUSSELDORF',
    5: 'HEATHROW',
    6: 'KASSEL',
    7: 'LJUBLJANA',
    8: 'MAASTRICHT',
    9: 'MADRID',
    10: 'MUNCHENB',
    11: 'OSLO',
    12: 'SONNBLICK',
    13: 'STOCKHOLM',
    14: 'VALENTIA'
}


In [89]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)

    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
    
    # Print unique labels for debugging
    print("Unique y_true_labels:", np.unique(y_true_labels))
    print("Unique y_pred_labels:", np.unique(y_pred_labels))

    # Map numeric labels to station names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    # Return the confusion matrix
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

# Call the updated confusion_matrix function
cm = confusion_matrix(y_test_one_hot, y_pred, stations)
print(cm)


Unique y_true_labels: [0]
Unique y_pred_labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Pred   BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                     
BASEL     59        33        66     280         275       162     300   

Pred   LJUBLJANA  MAASTRICHT  MADRID  MUNCHENB  OSLO  SONNBLICK  STOCKHOLM  \
True                                                                         
BASEL          4         126     164       991     4         13        731   

Pred   VALENTIA  
True             
BASEL      1382  
